In [1]:
from pathlib import Path
import imageio
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly.express as px
import matplotlib.pyplot as plt
from skimage import segmentation

dataset = [Path(*f.parts[-2:]) for f in Path("unmarked").glob("**/*.png") if f.parts[-2] != "02" and f.parts[-2] != "01"]
del dataset[-2]

def get_picture(pic):
    picture = imageio.imread(Path("unmarked") / pic)
    return picture

def get_center(pic):
    x, y = Path("marks", "centers", str(pic)+".center").open().read().split(", ")
    x, y = int(x), int(y)
    return (x, y)

def get_mask(pic):
    f = Path("marks", "masks", str(pic)+".binary")
    return imageio.imread(f).astype("bool")

In [4]:
from src.methods.aorta_contour import by_point

pictures = [get_picture(pic) for pic in dataset]
inners = [by_point(picture, get_center(pic)) for pic, picture in zip(dataset, pictures)]
labels = [get_mask(pic) for pic in dataset]
outers = [label.copy() for label in labels]
for i in range(len(pictures)):
    outers[i][inners[i]] = False

In [6]:
points = [pic[outer] for pic, outer in zip(pictures, outers)]

In [28]:
draw_object = {'color': [], 'study': []}
for pic, colors in zip(dataset, points):
    draw_object['color'].extend(colors)
    draw_object['study'] += [pic for i in colors]

px.histogram(draw_object, x="color", color='study', barmode="group", nbins=10)

In [24]:
help(px.histogram)

Help on function histogram in module plotly.express._chart_types:

histogram(data_frame=None, x=None, y=None, color=None, facet_row=None, facet_col=None, facet_col_wrap=0, hover_name=None, hover_data=None, animation_frame=None, animation_group=None, category_orders={}, labels={}, color_discrete_sequence=None, color_discrete_map={}, marginal=None, opacity=None, orientation='v', barmode='relative', barnorm=None, histnorm=None, log_x=False, log_y=False, range_x=None, range_y=None, histfunc=None, cumulative=None, nbins=None, title=None, template=None, width=None, height=None)
        In a histogram, rows of `data_frame` are grouped together into a
        rectangular mark to visualize the 1D distribution of an aggregate
        function `histfunc` (e.g. the count or sum) of the value `y` (or `x` if
        `orientation` is `'h'`).
        
    Parameters
    ----------
    data_frame: DataFrame or array-like or dict
        This argument needs to be passed for column names (and not keyword

In [13]:
df = px.data.tips()
fig = px.histogram(df, x="total_bill")
fig.show()